# Non-realtime Sonification

## Initialization

In [ ]:
import sc3nb as scn

# start scsynth
sc = scn.startup()

# connect supercollider server to system output
!jack_connect "SuperCollider:out_1" "system:playback_1"
!jack_connect "SuperCollider:out_2" "system:playback_2"

In [ ]:
# sc.exit()

In [ ]:
# test supercollider output
sc.server.blip()

In [ ]:
sc.server.latency = 0.1

In [ ]:
import os

# name given to the container
CONTAINER_NAME = 'openface'

# base directory of the container
CONTAINER_BASE_DIR = '/home/openface-build'
# directory with executalbles in the container
CONTAINER_BIN_DIR = os.path.join(CONTAINER_BASE_DIR, 'build/bin')

CONTAINER_FILE_DIR = os.path.join(CONTAINER_BASE_DIR, 'files')
CONTAINER_OUT_DIR  = os.path.join(CONTAINER_FILE_DIR, 'processed')

CONTAINER_EXECUTABLE = os.path.join(CONTAINER_BIN_DIR, 'FeatureExtraction')

# mounted local directories
FILE_DIR = 'mount'


def feature_extraction_offline(video_name):
    """Perform feature extraction on video file."""
    
    video_path = os.path.join(FILE_DIR, video_name)
    
    # the file must be in FILE_DIR
    if not os.path.isfile(video_path):
        raise FileNotFoundError(video_path)
    
    container_video_path = os.path.join(CONTAINER_FILE_DIR, video_name)
    
    command = [
        'docker', 'exec', CONTAINER_NAME, CONTAINER_EXECUTABLE,
        '-f', container_video_path,
        '-out_dir', CONTAINER_OUT_DIR,
        # features extracted
        '-pose', '-gaze', '-aus',
        # output tracked video
        '-tracked'
    ]
    
    # capture and combine stdout and stderr into one stream and set as text stream
    proc = subprocess.Popen(command,stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
        
    # poll process and show its output
    while True:
        output = proc.stdout.readline()
        
        if output:
            print(output.strip())
            
        if proc.poll() is not None:
            break
    
    return proc

When executing **offline feature extraction**, sometimes OpenFace will drop some frames without processing them, causing the output data (and tracked video output) to be inaccurate. This may be related to OpenCV not being able to work with some of the codecs, but in general this is an open issue of OpenFace.

One workaround is to:
1. Split video frames into separate images
    * `ffmpeg -i video.avi video_dir/frame%04d.jpg`
2. Instruct OpenFace to process the frame directory with the `-fdir` option
    * The csv output of openface will not contain timestamp information, so we would have to recreate them from frame files names

Here we will not be using this workaround.

The following functions are used to perform operations on video files using `ffmpeg`.

In [ ]:
# convert video file from one format to the other
!ffmpeg -y -i in_file -c:v libx264 -crf 22 -pix_fmt yuv420p -c:a libvo_aacenc -b:a 128k out_file

In [ ]:
# merge audio and video files
!ffmpeg -y -i video_file -i audio_file -map 0:v -map 1:a -c:v copy out_file
# ffmpeg -i files/phone-processed.mp4 -i score.wav  -c:v copy phone-processed-son.mp4 -y

## Load data

In this way it is possible to load the feature extraction data produced by OpenFace executables.

In [ ]:
import pandas as pd

df = pd.read_csv("mount/processed/phone.csv", sep=r',\s*', engine='python')
df.head()

## Sonification: AU04 Test

This sonification is similar to the previous sonifications, but with the addition of a parameter for amplitude regulation.

This is a very simple sonification of AU04 (Brow Lowerer). The intensity of AU04 is used here to modulate both the amplitude and the frequency of a continuous synth. As continuous synth, the default synth of sc3nb s2 is used (we will have to instruct the server to load it).

* The intensity range \[0,1\] is mapped into the amplitude range \[0,0.3\], where 0.3 will be the maximum amplitude of the sound. The sonification has a parameter amp that can be used to scale this range.
* The intensity range \[0,5\] is mapped into the midi range \[69,81\]

In [ ]:
import panson as ps
from panson import bundle

class AU04ContinuousSonification(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
        
    def init_parameters(self):
        self.amp = 1
    
    @bundle
    def init_server(self):
        self.s.load_synthdefs()

    @bundle
    def start(self):
        # lag time is decided based on the frame rate
        self.synth = scn.Synth("s2", {"amp": 0, "lg": 0.03})

    @bundle
    def _process(self, row):  
        self.synth.set(
            # only "max" should be enough (to clip the top part to 0.3)
            "amp", self.amp * scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax"),
            # map the intensity of the AU in one octave range
            "freq", scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))
        )

In [ ]:
son = AU04ContinuousSonification()
son

In [ ]:
import matplotlib
matplotlib.use('widget')

# %matplotlib widget

In [ ]:
vp = ps.VideoPlayer('mount/processed/phone.avi', fps=30)

In [ ]:
feature_display = ps.RTFeatureDisplay(['AU04_r', 'AU12_r'], queue_size=50)
dp = ps.DataPlayer(son, feature_display=feature_display, video_player=vp).load(df)

In [ ]:
feature_display.show(fps=30)
display(son)
display(dp)

In [ ]:
vp.quit()